In [1]:
# create dataset

from dataset_creator import create_dataset, MISSING_SYMBOLS
filename = create_dataset(dataset_size = 100, missing_percentage = 0.1, output_file="dataset.csv")

In [2]:
# read data
import numpy as np
import pandas as pd

data = pd.read_csv(filename)
data.head()

,name,surname,birthdate,results1,results2,category,email
0,Lance,Hutchinson,1923-10-02,40,0.28083643290350435,B,iwheeler@example.com
1,Cheryl,Brewer,1927-09-24,35,0.6318093648498969,C,perrydebbie@example.com
2,Derrick,Newton,2009-09-25,100,0.49056650688708503,C,malik95@example.org
3,John,Hamilton,1955-08-23,45,0.6184771382914842,A,foxzachary@example.org
4,Brandon,Jones,2021-11-24,13,na,none,NaN


In [3]:
dataset_scores = dict()

## Check if there are any missing values

In [4]:
data.replace(MISSING_SYMBOLS, np.nan, inplace=True)

In [5]:
data.isna().sum()

name         10
surname      10
birthdate    10
results1     10
results2     10
category     10
email        10
dtype: int64

In [6]:
dataset_scores["missing_percentage"] = data.isna().sum().sum()/data.size
dataset_scores["most_missing_column"] = data.isna().sum().max()/data.shape[0]

Other ideas: correlation, not good dates, mishmashed formats, upper and lower cased...

# Create badges

In [7]:
# save obtained scores to json
import json
 
filename="./badge_data.json"
json_object = json.dumps(dataset_scores, indent=4)
 
with open(filename, "w") as outfile:
    outfile.write(json_object)

In [8]:
repo_url = "https://github.com/annapanfil/data_quality_labeler" #todo: get dinamically

ownername = repo_url.split("/")[3]
repo_name = repo_url.split("/")[4]
branch = "main"


print("To add badges paste this to your readme.md file:")
for badge in dataset_scores.keys():
    print(f"![DQ Badge](https://img.shields.io/badge/dynamic/json?url=https%3A%2F%2Fraw.githubusercontent.com%2F{ownername}%2F{repo_name}%2F{branch}%2F{filename}&query=%24.{badge}&label={badge})")


To add badges paste this to your readme.md file:
![DQ Badge](https://img.shields.io/badge/dynamic/json?url=https%3A%2F%2Fraw.githubusercontent.com%2Fannapanfil%2Fdata_quality_labeler%2Fmain%2F./badge_data.json&query=%24.missing_percentage&label=missing_percentage)
![DQ Badge](https://img.shields.io/badge/dynamic/json?url=https%3A%2F%2Fraw.githubusercontent.com%2Fannapanfil%2Fdata_quality_labeler%2Fmain%2F./badge_data.json&query=%24.most_missing_column&label=most_missing_column)
